<a href="https://colab.research.google.com/github/varshanaradasu/Agentic-AI/blob/main/Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install langchain langchain-community langchain-google-genai langchain-chroma pypdf

In [38]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

In [39]:
loader=PyPDFLoader("file.pdf")
docs=loader.load()
docs

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-07-07T11:23:18+05:30', 'author': 'JHANSI LAXMI', 'moddate': '2025-07-07T11:23:18+05:30', 'source': 'file.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='Department of Computer Science and Engineering \nYear & Section: III &       Academic Year: 2025-26 \nMACHINE LEARNING LAB MANUAL \nLAB MANUAL – MODULE 1: Supervised Learning & Regression \n \nEXPERIMENT 1: Data Preprocessing and Feature Engineering \nProblem Statement: Given the Titanic Dataset, clean and preprocess the data to make it suitable for \nclassification. \nTasks: \n• Load the dataset using Pandas. \n• Identify and handle missing values using appropriate strategies. \n• Encode categorical variables using Label Encoding and One-Hot Encoding. \n• Split the dataset into train and test sets (80:20). \n• Apply feature scaling (StandardScaler or MinMaxScaler). \nExpected Outcome: \n• Cleaned datase

In [40]:
splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks=splitter.split_documents(docs)
print(len(chunks))
print(chunks[0].page_content)

9
Department of Computer Science and Engineering 
Year & Section: III &       Academic Year: 2025-26 
MACHINE LEARNING LAB MANUAL 
LAB MANUAL – MODULE 1: Supervised Learning & Regression 
 
EXPERIMENT 1: Data Preprocessing and Feature Engineering 
Problem Statement: Given the Titanic Dataset, clean and preprocess the data to make it suitable for 
classification. 
Tasks: 
• Load the dataset using Pandas. 
• Identify and handle missing values using appropriate strategies. 
• Encode categorical variables using Label Encoding and One-Hot Encoding. 
• Split the dataset into train and test sets (80:20). 
• Apply feature scaling (StandardScaler or MinMaxScaler). 
Expected Outcome: 
• Cleaned dataset ready for classification 
• Report shapes of original and processed datasets 
• Visualizations of missing data before and after cleaning 
 
EXPERIMENT 2: Regression Modeling for House Price Prediction 
Dataset: House Prices - Advanced Regression Techniques


In [41]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
embeddings=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001",
google_api_key=os.environ["GOOGLE_API_KEY"])

In [42]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks = splitter.split_documents(docs)
print(len(chunks))
print(chunks[0].page_content)


9
Department of Computer Science and Engineering 
Year & Section: III &       Academic Year: 2025-26 
MACHINE LEARNING LAB MANUAL 
LAB MANUAL – MODULE 1: Supervised Learning & Regression 
 
EXPERIMENT 1: Data Preprocessing and Feature Engineering 
Problem Statement: Given the Titanic Dataset, clean and preprocess the data to make it suitable for 
classification. 
Tasks: 
• Load the dataset using Pandas. 
• Identify and handle missing values using appropriate strategies. 
• Encode categorical variables using Label Encoding and One-Hot Encoding. 
• Split the dataset into train and test sets (80:20). 
• Apply feature scaling (StandardScaler or MinMaxScaler). 
Expected Outcome: 
• Cleaned dataset ready for classification 
• Report shapes of original and processed datasets 
• Visualizations of missing data before and after cleaning 
 
EXPERIMENT 2: Regression Modeling for House Price Prediction 
Dataset: House Prices - Advanced Regression Techniques


In [43]:
vector_store=Chroma.from_documents(documents=chunks,embedding=embeddings)

In [44]:
retriever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k":5})

In [45]:
chat_model=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.9,
    max_output_tokens=500
    )

In [46]:
prompt=PromptTemplate(
    template="""
    You are a helpful assistant.
    Answer from the provided transcript context..

    {context}
    Question: {question}
    """,
    input_variables=["context","question"]
)

In [47]:
question = 'what are tasks for experiment 3'
retrieved_docs = retriever.invoke(question)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [48]:
final_prompt = prompt.invoke({
    "context": context,
    "question": question
})
final_prompt

StringPromptValue(text='\n    You are a helpful assistant.\n    Answer from the provided transcript context..\n\n    EXPERIMENT 3: Heart Disease Classification Using Logistic Regression \nDataset: Heart Disease UCI Dataset \nProblem Statement: Predict whether a patient is likely to have heart disease. \nModels to Implement: \n• Logistic Regression \nTasks: \n• Train the model using 4 validation strategies: \no Simple hold-out validation \no K-fold cross validation \no Stratified K-fold cross validation \no Leave-One-Out (LOO) validation \n• Evaluate performance with Accuracy, Precision, Recall, F1 Score \n• Plot the confusion matrix \nExpected Outcome: \n• Tabulate and compare validation scores \n• Graph performance metrics and confusion matrix \n \nEXPERIMENT 4: Feature Selection on a Breast Cancer Dataset \nDataset: Breast Cancer Wisconsin Dataset \nProblem Statement: Select the most informative features to predict cancer diagnosis. \nTasks: \n• Apply Filter Method: Chi-Square test \

In [49]:
parser = StrOutputParser()

# Generate the answer
response = chat_model.invoke(final_prompt)

parser.invoke(response.content)

'The tasks for Experiment 3 are:\n\n*   Train the model using 4 validation strategies:\n    *   Simple hold-out validation\n    *   K-fold cross validation\n    *   Stratified K-fold cross validation\n    *   Leave-One-Out (LOO) validation\n*   Evaluate performance with Accuracy, Precision, Recall, F1 Score\n*   Plot the confusion matrix'